#### НОВАЯ АРХИТЕКТУРА МАППИНГА

#### Версия 5,6

In [ ]:
import os
import re
import json
import time
import pandas as pd
from collections import Counter
from ast import literal_eval
import requests
import numpy as np
import urllib3
import backoff
import math
from datetime import datetime

api_url = "https://api.mistral.ai/v1/chat/completions"
model_name = "mistral-small-latest"
api_key = ''
HEADERS = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}
# Отключаем предупреждения HTTPS
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ================== ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ==================

def safe_literal_eval(x):
    if isinstance(x, list):
        return x
    elif isinstance(x, str):
        try:
            return literal_eval(x)
        except:
            return []
    else:
        return []

def extract_json_from_markdown(content):
    """Извлекает JSON из markdown формата с бэктиками"""
    # Убираем бэктики и маркдаун
    content = re.sub(r'^```json\s*', '', content.strip())
    content = re.sub(r'^```\s*', '', content)
    content = re.sub(r'```$', '', content)
    return content.strip()

def safe_parse_json(content: str) -> dict:
    # Извлекаем JSON из markdown если нужно
    content = extract_json_from_markdown(content)
    
    # Приводим кавычки к стандартным
    content = content.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")
    # Убираем невидимые символы
    content = re.sub(r"[\u200B-\u200D\uFEFF]", "", content)

    try:
        # Убираем дубликаты ключей (оставляем последнее)
        def no_duplicate_pairs(pairs):
            d = {}
            for k, v in pairs:
                d[k] = v
            return d

        return json.loads(content, object_pairs_hook=no_duplicate_pairs)
    except Exception as e:
        print("⚠️ Не удалось распарсить JSON:", e)
        print("📋 Ответ как есть (первые 500 символов):", content[:500], "…")
        return {"anchors": [], "mapping": {}}

@backoff.on_exception(backoff.expo, 
                     (requests.exceptions.RequestException, ValueError),
                     max_tries=5,
                     max_time=60)
def send_request_to_llm(api_url, prompt, model_name, api_key, temperature=0.2, max_tokens=8192, delay=20):
    time.sleep(delay)
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": "Ты — ИИ для обработки и структурирования навыков в IT-рекрутинге. Отвечай ТОЛЬКО JSON без объяснений."},
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "response_format": {"type": "json_object"}  # Принудительно запрашиваем JSON
    }
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    
    try:
        resp = requests.post(api_url, json=payload, headers=headers, timeout=120, verify=False)
        
        if resp.status_code == 429:
            # Rate limiting - ждем подольше
            wait_time = 10
            print(f"⏳ Rate limit exceeded, waiting {wait_time} seconds...")
            time.sleep(wait_time)
            raise requests.exceptions.RequestException("Rate limit exceeded")
        
        if resp.status_code != 200:
            raise ValueError(f"Ошибка API: {resp.status_code} {resp.text}")
        
        resp_json = resp.json()
        content = resp_json["choices"][0]["message"]["content"]
        
        if not content or not content.strip():
            print("⚠️ Модель вернула пустой content")
            return {}
        
        return safe_parse_json(content)
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Ошибка сети: {e}")
        raise
    except Exception as e:
        print(f"❌ Неожиданная ошибка: {e}")
        raise

def call_llm(api_url, prompt, model_name, api_key, retries=3):
    for i in range(retries):
        try:
            print(f"🔄 Попытка {i+1} для запроса к LLM")
            data = send_request_to_llm(api_url, prompt, model_name, api_key)
            
            if isinstance(data, dict) and data:
                print(f"✅ Успешный JSON ответ (попытка {i+1})")
                return data
            else:
                print(f"⚠️ Пустой или невалидный JSON (попытка {i+1})")
                
        except Exception as e:
            print(f"❌ Ошибка в call_llm (попытка {i+1}): {e}")
            if i == retries - 1:  # Последняя попытка
                return {}
        
        # Увеличиваем задержку между попытками
        time.sleep(2 * (i + 1))
    
    print(f"❌ Все {retries} попыток провалились")
    return {}

# ================== ОСНОВНЫЕ ЭТАПЫ ==================

# 1. Чанк → якори + маппинг
def discover_chunk_anchors_and_map(position, chunk_skills, n, api_url, model_name, api_key):
    prompt = f"""
Профессия: {position}
Навыки: {chunk_skills}

1. Выдели до {n} якорей (категорий) только из этих навыков. 
   - Якоря должны быть конкретными технологиями или инструментами (SQL, Python, Tableau) по заданной професии
   - Избегай общих понятий (Анализ, Данные, Программирование)
   - Форма должна быть каноничной (короткая, общепринятая)
   - НЕ ПОВТОРЯЙ одинаковые якоря
2. Замапь все навыки из списка к выбранным якорям, если они являются синонимами. 
   - Если навык не подходит ни к одному якорю — оставь его неизменным
   - Возможны случаи , что у якоря нет синонимов 

ВЕРНИ СТРОГО В ФОРМАТЕ JSON БЕЗ ЛИШНИХ СИМВОЛОВ И БЕЗ ПОВТОРЕНИЙ.
Формат ответа ДОЛЖЕН БЫТЬ:
{{
  "anchors": ["якорь1", "якорь2", ...],
  "mapping": {{"skill1": "anchor1", "skill2": "skill2"}}
}}
"""
    response = call_llm(api_url, prompt, model_name, api_key)
    anchors = response.get("anchors", [])
    mapping = response.get("mapping", {})

    # 🔑 страховка: все навыки из чанка должны быть в mapping
    for s in chunk_skills:
        if s not in mapping:
            mapping[s] = s

    return anchors, mapping

# 2. Глобальная нормализация якорей (с добавленной защитой)
def normalize_global_anchors(position, all_local_anchors, api_url, model_name, api_key):
    if not all_local_anchors:
        print(f"⚠️  Нет локальных якорей для {position}")
        return [], {}
    
    # Если якорей мало, просто возвращаем их как есть
    if len(all_local_anchors) <= 10:
        base_anchors = list(set(all_local_anchors))
        mapping = {k: k for k in all_local_anchors}
        return base_anchors, mapping
    
    prompt = f"""
Для професии {position} собраны локальные якори-навыки (skills) : {all_local_anchors}

1. Выдели среди них базовые каноничные скилы base_anchors професии (с общепринятыми названиями).
2. Сгруппируй синонимы к этим базовым, если синонимы являются очень близкими по смыслу вариантами базового навыка или вариантами написания базового навыка.

Важно:
- Канонические скилы выбираются только из скиска всех якорей-навыков, не придумывай новые
- Если у базового якоря нет синонима - верни как есть.
- Каждый локальный якорь должен присутствовать в mapping.
- ВЕРНИ СТРОГО В ФОРМАТЕ JSON БЕЗ ЛИШНИХ СИМВОЛОВ.

Формат ответа ДОЛЖЕН БЫТЬ:
{{
  "base_anchors": ["base_anchor1", "base_anchor2", ...],
  "mapping": {{"локальный_якорь1": "base_anchor1", "локальный_якорь2": "base_anchor2", "локальный_якорь3": "base_anchor2", "локальный_якорь4": "локальный_якорь4",...}}
}}
"""
    
    try:
        response = call_llm(api_url, prompt, model_name, api_key)
        
        base_anchors = response.get("base_anchors", [])
        mapping = response.get("mapping", {})
        
        if not base_anchors:
            print(f"⚠️  Пустые base_anchors для {position}, используем fallback")
            base_anchors = list(set(all_local_anchors))
            mapping = {k: k for k in all_local_anchors}
        
        # Убедимся, что все локальные якори есть в mapping
        for anchor in all_local_anchors:
            if anchor not in mapping:
                mapping[anchor] = anchor
        
        print(f"✅ Final base_anchors для {position}: {len(base_anchors)} якорей")
        return base_anchors, mapping
        
    except Exception as e:
        print(f"❌ Ошибка в normalize_global_anchors для {position}: {e}")
        # Fallback: используем уникальные локальные якори
        base_anchors = list(set(all_local_anchors))
        mapping = {k: k for k in all_local_anchors}
        return base_anchors, mapping

# 3. Маппинг оставшихся скиллов к базовым якорям
def map_remaining_skills_to_base(position, base_anchors, remaining_skills, api_url, model_name, api_key, chunk_size=300):
    final_mapping = {}
    chunks = [remaining_skills[i:i+chunk_size] for i in range(0, len(remaining_skills), chunk_size)]
    
    for i, chunk in enumerate(chunks, 1):
        print(f"📦 Обрабатываю чанк {i}/{len(chunks)} для {position} ({len(chunk)} навыков)")
        
        prompt = f"""
Профессия: {position}
Базовые якори-навыки: {base_anchors}

Навыки: {chunk}

Замапь навыки к базовым якорям, если они являются синонимами или вариантами навыка близкими по смыслу.
Если навык не подходит ни к одному якорю — замапь его к 'Other'.
ВЕРНИ СТРОГО В ФОРМАТЕ JSON БЕЗ ЛИШНИХ СИМВОЛОВ.

Формат ответа ДОЛЖЕН БЫТЬ:
{{
  "mapping": {{"skill1": "anchor/Other", ...}}
}}
"""
        response = call_llm(api_url, prompt, model_name, api_key)
        mapping = response.get("mapping", {})

        # 🔑 страховка: все скиллы из чанка должны попасть в mapping
        for s in chunk:
            if s not in mapping:
                mapping[s] = s

        final_mapping.update(mapping)
    
    return final_mapping

# 4. Полный процесс для одной позиции
def process_position_pipeline(position, skills, n, api_url, model_name, api_key, chunk_size=200):
    # Считаем частоту навыков
    skill_counts = {}
    for skill in skills:
        # Пропускаем вложенные списки
        if isinstance(skill, list):
            print(f"⚠️  Пропускаем вложенный список: {skill}")
            continue
        skill_counts[skill] = skill_counts.get(skill, 0) + 1
    
    # Сортируем навыки по убыванию частоты
    unique_skills = sorted(skill_counts, key=skill_counts.get, reverse=True)
    
     
    print(f"🎯 Начинаю обработку {position}: {len(unique_skills)} уникальных навыков")
    
    chunks = [unique_skills[i:i+chunk_size] for i in range(0, len(unique_skills), chunk_size)]
    
    all_local_anchors, all_local_mapping = [], {}
    for i, chunk in enumerate(chunks, 1):
        print(f"🔍 Обрабатываю чанк {i}/{len(chunks)} для {position}")
        anchors, mapping = discover_chunk_anchors_and_map(position, chunk, n, api_url, model_name, api_key)
        all_local_anchors.extend(anchors)
        all_local_mapping.update(mapping)

    base_anchors, anchor_mapping = normalize_global_anchors(position, all_local_anchors, api_url, model_name, api_key)
    
    # ДОБАВЛЕНО: защита от пустых base_anchors
    if not base_anchors:
        base_anchors = list(set(all_local_anchors))
        anchor_mapping = {k: k for k in all_local_anchors}

    # нормализуем все маппинги
    normalized_mapping = {}
    for k, v in all_local_mapping.items():
        normalized_mapping[k] = anchor_mapping.get(v, v)

    # Находим навыки, которые нужно дополнительно обработать
    remaining_skills = []
    for skill in unique_skills:
        mapped_value = normalized_mapping.get(skill, skill)
        if mapped_value not in base_anchors and mapped_value == skill:
            remaining_skills.append(skill)

    # маппим остатки
    extra_mapping = {}
    if remaining_skills:
        print(f"📋 Дополнительная обработка {len(remaining_skills)} навыков для {position}")
        extra_mapping = map_remaining_skills_to_base(position, base_anchors, remaining_skills, api_url, model_name, api_key)

    # объединяем все маппинги
    final_mapping = {**normalized_mapping, **extra_mapping}

    # страховка: каждый навык должен остаться
    for s in unique_skills:
        if s not in final_mapping:
            final_mapping[s] = s

    # Статистика
    stats = {
        "total_skills": len(unique_skills),
        "mapped": sum(1 for v in final_mapping.values() if v in base_anchors),
        "unmapped": sum(1 for k, v in final_mapping.items() if v == k),
        "base_anchors": len(base_anchors),
        "remaining_sent": len(remaining_skills)
    }

    return final_mapping, base_anchors, stats

# 5. Обработка всех позиций в датафрейме
def process_all_positions(df, api_url, model_name, position_to_n, output_file, api_key, chunk_size=200):
    results = []
    positions = df["position"].unique()
    
    for i, pos in enumerate(positions, 1):
        print(f"\n{'='*50}")
        print(f"🚀 Обработка позиции {i}/{len(positions)}: {pos}")
        print(f"{'='*50}")
        
        all_skills = []
        for sublist in df.loc[df["position"] == pos, "skills"]:
            all_skills.extend(safe_literal_eval(sublist))

        n = position_to_n.get(pos, 30)  # Уменьшил дефолтное значение для экономии токенов
        
        try:
            mapping, anchors, stats = process_position_pipeline(pos, all_skills, n, api_url, model_name, api_key, chunk_size)
            
            results.append({
                "position": pos,
                "anchors": json.dumps(anchors, ensure_ascii=False),
                "mapping": json.dumps(mapping, ensure_ascii=False),
                "stats": json.dumps(stats, ensure_ascii=False)
            })
            
            print(f"✅ {pos}: {stats}")
            
            # Сохраняем промежуточные результаты после каждой позиции
            temp_df = pd.DataFrame(results)
            temp_df.to_csv(f"temp_1_{output_file}", index=False)
            
        except Exception as e:
            print(f"❌ Критическая ошибка при обработке {pos}: {e}")
            # Сохраняем то, что успели обработать
            if results:
                temp_df = pd.DataFrame(results)
                temp_df.to_csv(f"partial_{output_file}", index=False)
            continue

    pd.DataFrame(results).to_csv(output_file, index=False)
    print(f"Готово. Сохранено в {output_file}")



In [ ]:
# задаём для каких позиций сколько категорий хотим оставить
position_to_n = {
    'Специалист технической поддержки': 100,
    'Системный администратор': 100,
    'Дизайнер, художник': 100,
    'Тестировщик': 100,
    'Аналитик':	75,
    'Руководитель проектов': 70,
    'Специалист по информационной безопасности': 100,
    'Системный аналитик': 100,
    'Менеджер продукта': 100,
    'Системный инженер': 100,
    'Бизнес-аналитик': 100,
    'BI-аналитик, аналитик данных': 100,
    'Сетевой инженер': 50,
    'Руководитель группы разработки': 50,
    'Методолог': 100,
    'Технический писатель': 50,
    'Дата-сайентист':	50,
    'Технический директор (CTO)': 30,
    'Продуктовый аналитик': 50,
    'Арт-директор, креативный директор':	50,
    'Руководитель отдела аналитики':	50,
    'Директор по информационным технологиям (CIO)':	50,
    'Программист, разработчик': 100,
    'DevOps-инженер': 75 
    
}


input_file = "data/stage_5_df_train_result.csv"

output_file = "stage_5_df_train_result_skills_mapping_V6_2.csv"
#partial_file = "skills_mapping_mistral_partial.csv"

# === загружаем датафрейм ===
df = pd.read_csv(input_file)
# === отфильтруем ==#
df = df[df['position'].isin(position_to_n.keys())]

df["skills"] = df["skills"].apply(safe_literal_eval)   # приводим к list

# === запускаем обработку ===
process_all_positions(
    df=df,
    api_url=api_url,
    model_name=model_name,
    position_to_n=position_to_n,
    output_file=output_file,
    #partial_file=partial_file,
    api_key=api_key,
    chunk_size=600 
)


🚀 Обработка позиции 1/2: DevOps-инженер
🎯 Начинаю обработку DevOps-инженер: 1878 уникальных навыков
🔍 Обрабатываю чанк 1/4 для DevOps-инженер
🔄 Попытка 1 для запроса к LLM
⚠️ Не удалось распарсить JSON: Unterminated string starting at: line 559 column 12 (char 22463)
📋 Ответ как есть (первые 500 символов): {
  "anchors": ["Terraform", "Ansible", "Helm", "Kubernetes", "OpenShift", "Docker", "Podman", "Prometheus", "Grafana", "Zabbix", "Bash", "Python", "Linux", "PostgreSQL", "MySQL", "Oracle Database", "Microsoft SQL Server", "GitLab CI", "Jenkins", "GitHub Actions", "NGINX", "ELK Stack", "GIT", "Apache Kafka", "MongoDB", "ClickHouse", "Tarantool", "Cassandra", "Redis", "Go", "AWS", "SQL", "GITLab", "SonarQube", "Allure", "Azure DevOps", "HAProxy", "JAVA", "ArgoCD", "Flux", "TCP", "JIRA", "Apache", …
✅ Успешный JSON ответ (попытка 1)
🔍 Обрабатываю чанк 2/4 для DevOps-инженер
🔄 Попытка 1 для запроса к LLM
⚠️ Не удалось распарсить JSON: Unterminated string starting at: line 668 column 5 